In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# **Extracting Features using Vision Transformer**

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from torchvision import models, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vit = models.vit_b_16(pretrained=True).to(device)
vit.eval()

feature_extractor = torch.nn.Sequential(
    vit.encoder,
    torch.nn.Identity()
)

vit.heads = torch.nn.Identity()

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 99.6MB/s]


In [ ]:
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = vit(input_tensor)
        features = features.view(-1)
    return features.cpu()

In [ ]:
splits = ["train", "val", "test"]

In [ ]:
source_base = "drive/MyDrive/thesis2025/split_dataset_new"
target_base = "drive/MyDrive/thesis2025/split_dataset_vit3"

In [ ]:
for split in splits:
    split_path = os.path.join(source_base, split)
    subject_ids = os.listdir(split_path)

    for subject_id in tqdm(subject_ids, desc=f"Processing {split}"):
        subject_path = os.path.join(split_path, subject_id)
        eeg_dir = os.path.join(subject_path, "eeg_stft_spectrogram2")
        audio_dir = os.path.join(subject_path, "audio_spectrogram")

        eeg_files = sorted([os.path.join(eeg_dir, f) for f in os.listdir(eeg_dir) if f.endswith(".png")])
        audio_files = sorted([os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith(".png")])

        eeg_embeddings = []
        for p in eeg_files:
            try:
                eeg_embeddings.append(extract_features(p))
            except Exception as e:
                print(f"Skipping EEG image {p}: {e}")

        audio_embeddings = []
        for p in audio_files:
            try:
                audio_embeddings.append(extract_features(p))
            except Exception as e:
                print(f"Skipping audio image {p}: {e}")

        if not eeg_embeddings or not audio_embeddings:
            print(f"Skipping subject {subject_id} due to insufficient embeddings.")
            continue

        eeg_stack = torch.stack(eeg_embeddings)
        audio_stack = torch.stack(audio_embeddings)

        eeg_len = eeg_stack.size(0)
        audio_len = audio_stack.size(0)

        if audio_len < eeg_len:
            padding = torch.zeros((eeg_len - audio_len, audio_stack.size(1)))
            audio_stack = torch.cat([audio_stack, padding], dim=0)
        elif eeg_len < audio_len:
            padding = torch.zeros((audio_len - eeg_len, eeg_stack.size(1)))
            eeg_stack = torch.cat([eeg_stack, padding], dim=0)

        combined = torch.cat([eeg_stack, audio_stack], dim=1)
        combined_mean = combined.mean(dim=0)

        save_dir = os.path.join(target_base, split, subject_id)
        os.makedirs(save_dir, exist_ok=True)

        np.save(os.path.join(save_dir, "eeg_embedding.npy"), eeg_stack.numpy())
        np.save(os.path.join(save_dir, "audio_embedding.npy"), audio_stack.numpy())


Processing test: 100%|██████████| 6/6 [01:51<00:00, 18.65s/it]


In [ ]:
tri = np.load("drive/MyDrive/thesis2025/split_dataset_vit3/train/02010006/eeg_embedding.npy")

In [ ]:
tri.shape

(29, 768)

# **1D EEG Signal Extraction**

In [ ]:
import torch
import torch.nn as nn

class EEG1DEncoder(nn.Module):
    def __init__(self, hidden_dim=256, embed_dim=512):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_dim, batch_first=True, bidirectional=True)
        self.proj = nn.Linear(hidden_dim * 2, embed_dim)

    def forward(self, x):
        # x: (batch=1, channels=1, time)
        x = self.cnn(x)               # (1, 128, time')
        x = x.permute(0, 2, 1)        # (1, time', 128)
        lstm_out, _ = self.lstm(x)    # (1, time', hidden_dim*2)
        pooled = lstm_out.mean(dim=1) # (1, hidden_dim*2)
        embedding = self.proj(pooled) # (1, embed_dim)
        return embedding.squeeze(0)   # (embed_dim,)


In [ ]:
from pathlib import Path
# Setup device and model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EEG1DEncoder(hidden_dim=128, embed_dim=512).to(device)
model.eval()

# Paths
base_input_dir = Path("/content/drive/My Drive/thesis2025/split_dataset")
base_output_dir = Path("/content/drive/My Drive/thesis2025/split_dataset_vit3")

splits = ['train', 'val', 'test']

In [ ]:
# Feature extraction loop
with torch.no_grad():
    for split in splits:
        input_split_dir = base_input_dir / split
        output_split_dir = base_output_dir / split

        subject_folders = [f for f in input_split_dir.iterdir() if f.is_dir()]
        print(f"\nProcessing split: {split} ({len(subject_folders)} subjects)")

        for subject_folder in tqdm(subject_folders, desc=f"{split} split"):
            subject_id = subject_folder.name
            input_path = subject_folder / f"{subject_id}_processed.npy"
            if not input_path.exists():
                continue

            # Load EEG data (29 channels x time)
            eeg = np.load(input_path)  # (29, T)

            # Normalize each channel
            eeg = (eeg - eeg.mean(axis=1, keepdims=True)) / (eeg.std(axis=1, keepdims=True) + 1e-6)

            channel_embeddings = []

            # Encode channel by channel
            for ch_idx in range(eeg.shape[0]):
                ch_data = eeg[ch_idx]  # (T,)
                ch_tensor = torch.tensor(ch_data, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # (1, 1, T)

                embedding = model(ch_tensor)  # (embed_dim,)
                channel_embeddings.append(embedding.cpu().numpy())

            # Stack embeddings: (29, embed_dim)
            channel_embeddings = np.stack(channel_embeddings)

            # Save embedding
            output_subject_dir = output_split_dir / subject_id
            output_subject_dir.mkdir(parents=True, exist_ok=True)
            np.save(output_subject_dir / "eeg1d_embedding.npy", channel_embeddings)


Processing split: train (26 subjects)


train split: 100%|██████████| 26/26 [01:16<00:00,  2.93s/it]



Processing split: val (6 subjects)


val split: 100%|██████████| 6/6 [00:17<00:00,  2.96s/it]



Processing split: test (6 subjects)


test split: 100%|██████████| 6/6 [00:18<00:00,  3.02s/it]


In [ ]:
tri = np.load("drive/MyDrive/thesis2025/split_dataset_vit3/train/02010010/eeg1d_embedding.npy")

In [ ]:
tri.shape

(29, 512)

# **Classifier Module**

## **Max Pooling**

In [ ]:
import torch
import torch.nn as nn

class ConvPoolReLUClassifier(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512, num_classes=2):
        super(ConvPoolReLUClassifier, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)  # 29 → 14
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(256 * 14, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)  # (batch, 2048, 29)
        x = self.conv1(x)      # (batch, 256, 29)
        x = self.pool(x)       # (batch, 256, 14)
        x = self.relu(x)

        x = x.view(x.size(0), -1)  # (batch, 256*14)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from glob import glob

In [ ]:
class EEGAudioDataset(Dataset):
    def __init__(self, subject_dirs):
        self.subject_dirs = subject_dirs

    def __len__(self):
        return len(self.subject_dirs)

    def __getitem__(self, idx):
        subject_path = self.subject_dirs[idx]
        subject_id = os.path.basename(subject_path)

        eeg_embedding = np.load(os.path.join(subject_path, "eeg_embedding.npy"))
        audio_embedding = np.load(os.path.join(subject_path, "audio_embedding.npy"))
        raw_eeg_embedding = np.load(os.path.join(subject_path, "eeg1d_embedding.npy"))

        combined_embedding = np.concatenate([eeg_embedding, audio_embedding, raw_eeg_embedding], axis=1)
        label = 1 if subject_id.startswith('0201') else 0

        combined_embedding = torch.tensor(combined_embedding, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        return combined_embedding, label


In [ ]:
# ---- Training ----
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=70):
    best_model = None
    best_val_acc = 0

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in tqdm(val_loader):
                x, y = x.to(device), y.to(device)
                output = model(x)
                preds = output.argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        val_acc = correct / total
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model


In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn_padd(batch):
    """
    batch: list of tuples (x, y)
    x: tensor with shape (seq_len, feat_dim)
    y: int label
    """
    sequences = [item[0] for item in batch]  # list of (seq_len, feat_dim) tensors
    labels = torch.tensor([item[1] for item in batch], dtype=torch.long)

    padded_seqs = pad_sequence(sequences, batch_first=True, padding_value=0)

    return padded_seqs, labels


In [ ]:
base_dir = "drive/MyDrive/thesis2025/split_dataset_vit3"

# Step 1: Index all subject folders from train, val, test
subject_path_map = {}
for split in ['train', 'val', 'test']:
    split_path = os.path.join(base_dir, split)
    for subj in os.listdir(split_path):
        subject_path_map[subj] = os.path.join(split_path, subj)

In [ ]:
import os
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
with open("drive/MyDrive/thesis2025/split_dataset_june/fold_assignments.json", "r") as f:
    folds = json.load(f)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx in range(5):
    fold_name = f"fold_{fold_idx + 1}"
    train_ids = folds[fold_name]["train"]
    val_ids   = folds[fold_name]["val"]
    test_ids  = folds[fold_name]["test"]

    train_subjs = [subject_path_map[sid] for sid in train_ids]
    val_subjs   = [subject_path_map[sid] for sid in val_ids]
    test_subjs  = [subject_path_map[sid] for sid in test_ids]

    train_set = EEGAudioDataset(train_subjs)
    val_set = EEGAudioDataset(val_subjs)
    test_set = EEGAudioDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=2048).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0004)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


100%|██████████| 1/1 [00:00<00:00, 42.64it/s]


Fold 1 Test Accuracy: 0.3750, Precision: 0.3667, Recall: 0.3750, F1: 0.3651


100%|██████████| 1/1 [00:00<00:00, 38.44it/s]


Fold 2 Test Accuracy: 0.6250, Precision: 0.6333, Recall: 0.6250, F1: 0.6190


100%|██████████| 1/1 [00:00<00:00, 44.62it/s]


Fold 3 Test Accuracy: 0.6250, Precision: 0.6250, Recall: 0.6333, F1: 0.6190


100%|██████████| 1/1 [00:00<00:00, 30.50it/s]


Fold 4 Test Accuracy: 0.8571, Precision: 0.9000, Recall: 0.8333, F1: 0.8444


100%|██████████| 1/1 [00:00<00:00, 32.58it/s]

Fold 5 Test Accuracy: 0.4286, Precision: 0.2500, Recall: 0.3750, F1: 0.3000

5-Fold CV Results:
Mean Accuracy  = 0.5821 ± 0.1708
Mean Precision = 0.5550 ± 0.2274
Mean Recall    = 0.5683 ± 0.1746
Mean F1-Score  = 0.5495 ± 0.1964


## **Adaptive Avg**

In [ ]:
import torch
import torch.nn as nn

class ConvPoolReLUClassifier(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=1024, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=256, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(256, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.pool(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
import os
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
with open("drive/MyDrive/thesis2025/split_dataset_june/fold_assignments.json", "r") as f:
    folds = json.load(f)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx in range(5):
    fold_name = f"fold_{fold_idx + 1}"
    train_ids = folds[fold_name]["train"]
    val_ids   = folds[fold_name]["val"]
    test_ids  = folds[fold_name]["test"]

    train_subjs = [subject_path_map[sid] for sid in train_ids]
    val_subjs   = [subject_path_map[sid] for sid in val_ids]
    test_subjs  = [subject_path_map[sid] for sid in test_ids]

    train_set = EEGAudioDataset(train_subjs)
    val_set = EEGAudioDataset(val_subjs)
    test_set = EEGAudioDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=2048).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0004)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


100%|██████████| 1/1 [00:00<00:00, 44.36it/s]


Fold 1 Test Accuracy: 0.2500, Precision: 0.2500, Recall: 0.2500, F1: 0.2500


100%|██████████| 1/1 [00:00<00:00, 26.40it/s]


Fold 2 Test Accuracy: 0.5000, Precision: 0.5000, Recall: 0.5000, F1: 0.4667


100%|██████████| 1/1 [00:00<00:00, 40.16it/s]


Fold 3 Test Accuracy: 0.7500, Precision: 0.7333, Recall: 0.7333, F1: 0.7333


100%|██████████| 1/1 [00:00<00:00, 31.48it/s]


Fold 4 Test Accuracy: 0.8571, Precision: 0.9000, Recall: 0.8333, F1: 0.8444


100%|██████████| 1/1 [00:00<00:00, 21.14it/s]

Fold 5 Test Accuracy: 0.4286, Precision: 0.2500, Recall: 0.3750, F1: 0.3000

5-Fold CV Results:
Mean Accuracy  = 0.5571 ± 0.2197
Mean Precision = 0.5267 ± 0.2592
Mean Recall    = 0.5383 ± 0.2174
Mean F1-Score  = 0.5189 ± 0.2345


## **GlobalAvg Pooling**

In [ ]:
import torch
import torch.nn as nn

class ConvPoolReLUClassifier(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=1024, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(256, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)        # (batch, input_dim, seq_len)
        x = self.conv1(x)            # (batch, 256, seq_len)
        x = self.relu(x)
        x = torch.mean(x, dim=2)     # Global average pooling across time dimension
        x = self.relu(self.fc1(x))   # (batch, hidden_dim)
        return self.fc2(x)           # (batch, num_classes)


In [ ]:
import os
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
with open("drive/MyDrive/thesis2025/split_dataset_june/fold_assignments.json", "r") as f:
    folds = json.load(f)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx in range(5):
    fold_name = f"fold_{fold_idx + 1}"
    train_ids = folds[fold_name]["train"]
    val_ids   = folds[fold_name]["val"]
    test_ids  = folds[fold_name]["test"]

    train_subjs = [subject_path_map[sid] for sid in train_ids]
    val_subjs   = [subject_path_map[sid] for sid in val_ids]
    test_subjs  = [subject_path_map[sid] for sid in test_ids]

    train_set = EEGAudioDataset(train_subjs)
    val_set = EEGAudioDataset(val_subjs)
    test_set = EEGAudioDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=2048).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0004)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


100%|██████████| 1/1 [00:00<00:00, 41.47it/s]


Fold 1 Test Accuracy: 0.2500, Precision: 0.2500, Recall: 0.2500, F1: 0.2500


100%|██████████| 1/1 [00:00<00:00, 40.48it/s]


Fold 2 Test Accuracy: 0.6250, Precision: 0.6333, Recall: 0.6250, F1: 0.6190


100%|██████████| 1/1 [00:00<00:00, 40.22it/s]


Fold 3 Test Accuracy: 0.7500, Precision: 0.7333, Recall: 0.7333, F1: 0.7333


100%|██████████| 1/1 [00:00<00:00, 33.83it/s]


Fold 4 Test Accuracy: 0.8571, Precision: 0.9000, Recall: 0.8333, F1: 0.8444


100%|██████████| 1/1 [00:00<00:00, 31.89it/s]

Fold 5 Test Accuracy: 0.4286, Precision: 0.2500, Recall: 0.3750, F1: 0.3000

5-Fold CV Results:
Mean Accuracy  = 0.5821 ± 0.2189
Mean Precision = 0.5533 ± 0.2619
Mean Recall    = 0.5633 ± 0.2187
Mean F1-Score  = 0.5494 ± 0.2356


## **Attention Pooling**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attn_weights = nn.Linear(input_dim, 1)

    def forward(self, x):
        # x: (batch, seq_len, dim)
        scores = self.attn_weights(x)             # (batch, seq_len, 1)
        weights = F.softmax(scores, dim=1)        # (batch, seq_len, 1)
        pooled = torch.sum(weights * x, dim=1)    # (batch, dim)
        return pooled

class ConvPoolReLUClassifier(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=1024, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.attn_pool = AttentionPooling(input_dim=256)
        self.fc1 = nn.Linear(256, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.transpose(1, 2)                    # (batch, input_dim, seq_len)
        x = self.conv1(x)                        # (batch, 256, seq_len)
        x = self.relu(x)
        x = x.transpose(1, 2)                    # (batch, seq_len, 256)
        x = self.attn_pool(x)                    # (batch, 256)
        x = self.relu(self.fc1(x))               # (batch, hidden_dim)
        return self.fc2(x)                       # (batch, num_classes)


In [ ]:
import os
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
with open("drive/MyDrive/thesis2025/split_dataset_june/fold_assignments.json", "r") as f:
    folds = json.load(f)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx in range(5):
    fold_name = f"fold_{fold_idx + 1}"
    train_ids = folds[fold_name]["train"]
    val_ids   = folds[fold_name]["val"]
    test_ids  = folds[fold_name]["test"]

    train_subjs = [subject_path_map[sid] for sid in train_ids]
    val_subjs   = [subject_path_map[sid] for sid in val_ids]
    test_subjs  = [subject_path_map[sid] for sid in test_ids]

    train_set = EEGAudioDataset(train_subjs)
    val_set = EEGAudioDataset(val_subjs)
    test_set = EEGAudioDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=2048).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0004)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


100%|██████████| 1/1 [00:00<00:00, 38.28it/s]


Fold 1 Test Accuracy: 0.2500, Precision: 0.2500, Recall: 0.2500, F1: 0.2500


100%|██████████| 1/1 [00:00<00:00, 43.12it/s]


Fold 2 Test Accuracy: 0.5000, Precision: 0.5000, Recall: 0.5000, F1: 0.4667


100%|██████████| 1/1 [00:00<00:00, 40.07it/s]


Fold 3 Test Accuracy: 0.6250, Precision: 0.6250, Recall: 0.6333, F1: 0.6190


100%|██████████| 1/1 [00:00<00:00, 20.20it/s]


Fold 4 Test Accuracy: 0.8571, Precision: 0.9000, Recall: 0.8333, F1: 0.8444


100%|██████████| 1/1 [00:00<00:00, 31.89it/s]

Fold 5 Test Accuracy: 0.4286, Precision: 0.2500, Recall: 0.3750, F1: 0.3000

5-Fold CV Results:
Mean Accuracy  = 0.5321 ± 0.2028
Mean Precision = 0.5050 ± 0.2452
Mean Recall    = 0.5183 ± 0.2026
Mean F1-Score  = 0.4960 ± 0.2174


# **Manual Hyperparams Tuning**

In [ ]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
with open("drive/MyDrive/thesis2025/split_dataset_june/fold_assignments.json", "r") as f:
    folds = json.load(f)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx in range(5):
    fold_name = f"fold_{fold_idx + 1}"
    train_ids = folds[fold_name]["train"]
    val_ids   = folds[fold_name]["val"]
    test_ids  = folds[fold_name]["test"]

    train_subjs = [subject_path_map[sid] for sid in train_ids]
    val_subjs   = [subject_path_map[sid] for sid in val_ids]
    test_subjs  = [subject_path_map[sid] for sid in test_ids]

    train_set = EEGAudioDataset(train_subjs)
    val_set = EEGAudioDataset(val_subjs)
    test_set = EEGAudioDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=2048).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


100%|██████████| 1/1 [00:00<00:00, 40.99it/s]


Fold 1 Test Accuracy: 0.3750, Precision: 0.3667, Recall: 0.3750, F1: 0.3651


100%|██████████| 1/1 [00:00<00:00, 40.58it/s]


Fold 2 Test Accuracy: 0.7500, Precision: 0.7500, Recall: 0.7500, F1: 0.7500


100%|██████████| 1/1 [00:00<00:00, 40.46it/s]


Fold 3 Test Accuracy: 0.7500, Precision: 0.7333, Recall: 0.7333, F1: 0.7333


100%|██████████| 1/1 [00:00<00:00, 31.92it/s]


Fold 4 Test Accuracy: 0.7143, Precision: 0.8333, Recall: 0.6667, F1: 0.6500


100%|██████████| 1/1 [00:00<00:00, 22.03it/s]

Fold 5 Test Accuracy: 0.4286, Precision: 0.2500, Recall: 0.3750, F1: 0.3000

5-Fold CV Results:
Mean Accuracy  = 0.6036 ± 0.1661
Mean Precision = 0.5867 ± 0.2327
Mean Recall    = 0.5800 ± 0.1697
Mean F1-Score  = 0.5597 ± 0.1897


# **Incorporating Text Modality**

In [ ]:
tri = np.load("drive/MyDrive/thesis2025/dataset_he/02010006/text_embedding.npy")

In [ ]:
tri.shape

(29, 768)

In [ ]:
from pathlib import Path
import shutil

source_base = Path("drive/MyDrive/thesis2025/dataset_he")
target_base = Path("drive/MyDrive/thesis2025/split_dataset_vit3")

splits = ["train", "val", "test"]

for split in splits:
    split_dir = target_base / split
    subject_dirs = [d for d in split_dir.iterdir() if d.is_dir()]

    print(f"Processing {split} split: {len(subject_dirs)} subjects")

    for subject_dir in subject_dirs:
        subject_id = subject_dir.name
        src_file = source_base / subject_id / "text_embedding.npy"
        dst_file = subject_dir / "text_embedding.npy"

        if src_file.exists():
            shutil.copy(src_file, dst_file)
        else:
            print(f"Warning: {src_file} does not exist!")


Processing train split: 26 subjects
Processing val split: 6 subjects
Processing test split: 6 subjects


In [ ]:
class EEGAudioTextDataset(Dataset):
    def __init__(self, subject_dirs):
        self.subject_dirs = subject_dirs

    def __len__(self):
        return len(self.subject_dirs)

    def __getitem__(self, idx):
        subject_path = self.subject_dirs[idx]
        subject_id = os.path.basename(subject_path)

        eeg_embedding = np.load(os.path.join(subject_path, "eeg_embedding.npy"))
        audio_embedding = np.load(os.path.join(subject_path, "audio_embedding.npy"))
        raw_eeg_embedding = np.load(os.path.join(subject_path, "eeg1d_embedding.npy"))
        text_embedding = np.load(os.path.join(subject_path, "text_embedding.npy"))

        combined_embedding = np.concatenate([eeg_embedding, audio_embedding, raw_eeg_embedding, text_embedding], axis=1)
        label = 1 if subject_id.startswith('0201') else 0

        combined_embedding = torch.tensor(combined_embedding, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        return combined_embedding, label

In [ ]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
with open("drive/MyDrive/thesis2025/split_dataset_june/fold_assignments.json", "r") as f:
    folds = json.load(f)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx in range(5):
    fold_name = f"fold_{fold_idx + 1}"
    train_ids = folds[fold_name]["train"]
    val_ids   = folds[fold_name]["val"]
    test_ids  = folds[fold_name]["test"]

    train_subjs = [subject_path_map[sid] for sid in train_ids]
    val_subjs   = [subject_path_map[sid] for sid in val_ids]
    test_subjs  = [subject_path_map[sid] for sid in test_ids]

    train_set = EEGAudioTextDataset(train_subjs)
    val_set = EEGAudioTextDataset(val_subjs)
    test_set = EEGAudioTextDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=2816).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


100%|██████████| 1/1 [00:00<00:00, 32.70it/s]


Fold 1 Test Accuracy: 0.3750, Precision: 0.3667, Recall: 0.3750, F1: 0.3651


100%|██████████| 1/1 [00:00<00:00, 21.41it/s]


Fold 2 Test Accuracy: 0.6250, Precision: 0.6333, Recall: 0.6250, F1: 0.6190


100%|██████████| 1/1 [00:00<00:00, 30.93it/s]


Fold 3 Test Accuracy: 0.8750, Precision: 0.9167, Recall: 0.8333, F1: 0.8545


100%|██████████| 1/1 [00:00<00:00, 21.97it/s]


Fold 4 Test Accuracy: 0.8571, Precision: 0.9000, Recall: 0.8333, F1: 0.8444


100%|██████████| 1/1 [00:00<00:00, 22.46it/s]

Fold 5 Test Accuracy: 0.4286, Precision: 0.2500, Recall: 0.3750, F1: 0.3000

5-Fold CV Results:
Mean Accuracy  = 0.6321 ± 0.2084
Mean Precision = 0.6133 ± 0.2711
Mean Recall    = 0.6083 ± 0.2051
Mean F1-Score  = 0.5966 ± 0.2324


# **Let's do Unimodal Text**

In [ ]:
class TextDataset(Dataset):
    def __init__(self, subject_dirs):
        self.subject_dirs = subject_dirs

    def __len__(self):
        return len(self.subject_dirs)

    def __getitem__(self, idx):
        subject_path = self.subject_dirs[idx]
        subject_id = os.path.basename(subject_path)

        text_embedding = np.load(os.path.join(subject_path, "text_embedding.npy"))
        label = 1 if subject_id.startswith('0201') else 0

        combined_embedding = torch.tensor(text_embedding, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        return combined_embedding, label

In [ ]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
base_dir = "drive/MyDrive/thesis2025/split_dataset_vit3"
split_dirs = ["train", "val", "test"]

all_subject_dirs = []
for split in split_dirs:
    all_subject_dirs.extend(sorted(glob(os.path.join(base_dir, split, "*"))))

labels = [1 if os.path.basename(d).startswith("0201") else 0 for d in all_subject_dirs]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx, (train_idx, test_idx) in enumerate(skf.split(all_subject_dirs, labels)):
    print(f"\nFold {fold_idx + 1}/5")
    train_subjs = [all_subject_dirs[i] for i in train_idx]
    test_subjs = [all_subject_dirs[i] for i in test_idx]
    train_subjs, val_subjs = train_test_split(train_subjs, test_size=0.1, random_state=42)

    train_set = TextDataset(train_subjs)
    val_set = TextDataset(val_subjs)
    test_set = TextDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=768).to(device)
    optimizer = optim.Adamax(model.parameters(), lr=0.0004, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


Fold 1/5


100%|██████████| 1/1 [00:00<00:00, 105.37it/s]


Fold 1 Test Accuracy: 0.8750, Precision: 0.9000, Recall: 0.8750, F1: 0.8730

Fold 2/5


100%|██████████| 1/1 [00:00<00:00, 107.81it/s]


Fold 2 Test Accuracy: 0.6250, Precision: 0.6333, Recall: 0.6250, F1: 0.6190

Fold 3/5


...

Fold 3 Test Accuracy: 0.7500, Precision: 0.8000, Recall: 0.8000, F1: 0.7500

Fold 4/5


100%|██████████| 1/1 [00:00<00:00, 87.08it/s]


Fold 4 Test Accuracy: 0.7143, Precision: 0.8333, Recall: 0.6667, F1: 0.6500

Fold 5/5


100%|██████████| 1/1 [00:00<00:00, 89.49it/s]


Fold 5 Test Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000

5-Fold CV Results:
Mean Accuracy  = 0.7929 ± 0.1310
Mean Precision = 0.8333 ± 0.1211
Mean Recall    = 0.7933 ± 0.1370
Mean F1-Score  = 0.7784 ± 0.1419


In [ ]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

set_seed(42)
# ---- Cross-Validation ----
base_dir = "drive/MyDrive/thesis2025/split_dataset_vit3"
split_dirs = ["train", "val", "test"]

all_subject_dirs = []
for split in split_dirs:
    all_subject_dirs.extend(sorted(glob(os.path.join(base_dir, split, "*"))))

labels = [1 if os.path.basename(d).startswith("0201") else 0 for d in all_subject_dirs]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []
fold_reports = []
fold_conf_matrices = []

for fold_idx, (train_idx, test_idx) in enumerate(skf.split(all_subject_dirs, labels)):
    print(f"\nFold {fold_idx + 1}/5")
    train_subjs = [all_subject_dirs[i] for i in train_idx]
    test_subjs = [all_subject_dirs[i] for i in test_idx]
    train_subjs, val_subjs = train_test_split(train_subjs, test_size=0.1, random_state=42)

    train_set = TextDataset(train_subjs)
    val_set = TextDataset(val_subjs)
    test_set = TextDataset(test_subjs)

    train_loader = DataLoader(train_set, batch_size=100, shuffle=True, collate_fn=collate_fn_padd)
    val_loader = DataLoader(val_set, batch_size=100, collate_fn=collate_fn_padd)
    test_loader = DataLoader(test_set, batch_size=100, collate_fn=collate_fn_padd)

    model = ConvPoolReLUClassifier(input_dim=768).to(device)
    optimizer = optim.Adamax(model.parameters(), lr=0.0004)
    criterion = nn.CrossEntropyLoss()

    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # ---- Evaluation ----
    trained_model.eval()
    correct = total = 0
    fold_preds = []
    fold_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = trained_model(x)
            preds = output.argmax(dim=1)

            fold_preds.extend(preds.cpu().numpy())
            fold_labels.extend(y.cpu().numpy())

            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    precision = precision_score(fold_labels, fold_preds, average='macro')
    recall = recall_score(fold_labels, fold_preds, average='macro')
    f1 = f1_score(fold_labels, fold_preds, average='macro')

    print(f"Fold {fold_idx + 1} Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    results.append((acc, precision, recall, f1))

    # Store classification report and confusion matrix for later averaging
    report = classification_report(fold_labels, fold_preds, output_dict=True, target_names=["Healthy", "Depressed"])
    conf_matrix = confusion_matrix(fold_labels, fold_preds)

    fold_reports.append(report)
    fold_conf_matrices.append(conf_matrix)

# ---- Final Report ----
results = np.array(results)
mean_acc, mean_prec, mean_rec, mean_f1 = results.mean(axis=0)
std_acc = results[:, 0].std()
std_prec = results[:, 1].std()
std_rec = results[:, 2].std()
std_f1 = results[:, 3].std()

print(f"\n5-Fold CV Results:")
print(f"Mean Accuracy  = {mean_acc:.4f} ± {std_acc:.4f}")
print(f"Mean Precision = {mean_prec:.4f} ± {std_prec:.4f}")
print(f"Mean Recall    = {mean_rec:.4f} ± {std_rec:.4f}")
print(f"Mean F1-Score  = {mean_f1:.4f} ± {std_f1:.4f}")


Fold 1/5


100%|██████████| 1/1 [00:00<00:00, 42.46it/s]


Fold 1 Test Accuracy: 0.8750, Precision: 0.9000, Recall: 0.8750, F1: 0.8730

Fold 2/5


100%|██████████| 1/1 [00:00<00:00, 42.39it/s]


Fold 2 Test Accuracy: 0.6250, Precision: 0.6333, Recall: 0.6250, F1: 0.6190

Fold 3/5


...

Fold 3 Test Accuracy: 0.7500, Precision: 0.8000, Recall: 0.8000, F1: 0.7500

Fold 4/5


100%|██████████| 1/1 [00:00<00:00, 41.08it/s]


Fold 4 Test Accuracy: 0.7143, Precision: 0.8333, Recall: 0.6667, F1: 0.6500

Fold 5/5


100%|██████████| 1/1 [00:00<00:00, 43.97it/s]

Fold 5 Test Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000

5-Fold CV Results:
Mean Accuracy  = 0.7929 ± 0.1310
Mean Precision = 0.8333 ± 0.1211
Mean Recall    = 0.7933 ± 0.1370
Mean F1-Score  = 0.7784 ± 0.1419
